For each year we will precalculate levels of gas leaks per person for each geoid. Then we will group geoids by levels and create geolayers. So, each layer will have a group of geoid with ~ same level of gas leaks per person

In [37]:
import plotly.offline as py
import plotly.graph_objs as graph_objs
import pandas as pd
import geopandas as gpd
import numpy as np
import re

mapbox_access_token = "pk.eyJ1IjoibWlzaGtpY2UiLCJhIjoiY2s5MG94bWRoMDQxdjNmcHI1aWI1YnFkYyJ9.eFsHqEMYY7qxa0Pb9USCtQ"

In [38]:
dfg = gpd.read_file('C:/Users/mskac/machineLearning/GasLeakConEd/data/processed/2010 Census Tracts with geoid.geojson')
dfg['geoid'] = dfg.block_fips.str[:11]
dfg = dfg[['geoid', 'ntaname', 'boro_name', 'shape_area', 'geometry']]
dfg.columns = map(str.lower, dfg.columns)
dfg = dfg.applymap(lambda s:s.lower() if type(s) == str else s)

In [39]:
dfg.head()

,geoid,ntaname,boro_name,shape_area,geometry
0,36085000900,west new brighton-new brighton-st. george,staten island,2497009.71359,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ..."
1,36061009800,turtle bay-east midtown,manhattan,1906016.35002,"MULTIPOLYGON (((-73.96433 40.75638, -73.96479 ..."
2,36061010000,turtle bay-east midtown,manhattan,1860938.37721,"MULTIPOLYGON (((-73.96802 40.75958, -73.96848 ..."
3,36061010200,midtown-midtown south,manhattan,1860992.70485,"MULTIPOLYGON (((-73.97124 40.76094, -73.97170 ..."
4,36061010400,midtown-midtown south,manhattan,1864600.37503,"MULTIPOLYGON (((-73.97446 40.76229, -73.97491 ..."


In [40]:
dfg[dfg['ntaname'].str.contains('cemetery')].head()

,geoid,ntaname,boro_name,shape_area,geometry
774,36061000100,park-cemetery-etc-manhattan,manhattan,1844421.23744,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ..."
889,36047017500,park-cemetery-etc-brooklyn,brooklyn,21101598.5644,"MULTIPOLYGON (((-73.99238 40.65890, -73.99019 ..."
1001,36081062400,park-cemetery-etc-queens,queens,5374590.6882,"MULTIPOLYGON (((-73.74125 40.68746, -73.73760 ..."
1226,36047015400,park-cemetery-etc-brooklyn,brooklyn,9969049.01193,"MULTIPOLYGON (((-74.02053 40.61806, -74.01984 ..."
1289,36047070203,park-cemetery-etc-brooklyn,brooklyn,43216571.4155,"MULTIPOLYGON (((-73.86706 40.58209, -73.86769 ..."


In [26]:
full_df = pd.read_csv('https://raw.githubusercontent.com/MarinaOrzechowski/GasLeakConEd/timeline_branch/data/processed/important_(used_in_app)/Merged_asc_fdny_data.csv')

In [27]:
full_df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [28]:
full_df = full_df.dropna()

In [29]:

full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11556 entries, 0 to 11606
Data columns (total 19 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   incident_year                                            11556 non-null  int64  
 1   geoid                                                    11556 non-null  int64  
 2   gas_leaks_per_person                                     11556 non-null  float64
 3   avg_assigned_fdny_engines#                               11556 non-null  float64
 4   avg_incident_duration                                    11556 non-null  float64
 5   median_age                                               11556 non-null  float64
 6   avg_houshold_size                                        11556 non-null  float64
 7   avg_year_built                                           11556 non-null  float64
 8   nonfamily_housholds%      

In [30]:
full_df[full_df.gas_leaks_per_person > 0.003310].describe()

,incident_year,geoid,gas_leaks_per_person,avg_assigned_fdny_engines#,avg_incident_duration,median_age,avg_houshold_size,avg_year_built,nonfamily_housholds%,lonely_housholder%,lonely_housholder_over65%,housholders_grandparents_responsible_for_grandchildren%,unemployed%,median_houshold_income,mean_houshold_income,total_housing_units,occupied_housing_units%,vacant_housing_units%,not_us_citizen%
count,2889.000000,2.889000e+03,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000
mean,2015.771201,3.604885e+10,0.007172,2.062730,1392.592602,35.876843,2.542378,1949.817780,43.387317,35.088445,11.002267,2.224546,10.436622,57330.690550,82635.638629,1823.581516,90.168847,9.831326,14.606438
std,1.559281,2.503189e+07,0.020157,0.197013,378.556565,6.532454,0.526167,13.676322,16.276726,13.111069,6.958534,3.071707,6.525277,36490.440626,57392.139721,1238.439008,7.634382,7.634398,7.352100
min,2013.000000,3.600500e+10,0.003310,0.714286,629.250000,15.700000,1.030000,1922.479141,0.000000,0.000000,0.000000,0.000000,0.000000,9053.000000,11652.000000,16.000000,16.100000,0.000000,0.000000
25%,2015.000000,3.604702e+10,0.003935,1.960000,1151.666667,31.700000,2.210000,1940.160480,31.300161,26.112760,6.688778,0.000000,5.700000,30893.000000,45552.000000,1043.000000,87.500000,5.200000,9.407070
50%,2016.000000,3.604711e+10,0.004911,2.060000,1338.250000,34.900000,2.550000,1949.230565,41.403126,33.333333,9.860724,1.454898,9.300000,46810.000000,65599.000000,1555.000000,91.600000,8.400000,13.370474
75%,2017.000000,3.606102e+10,0.006981,2.166667,1539.035714,38.900000,2.900000,1957.333523,53.057065,42.257742,13.907285,3.426791,13.900000,74712.000000,98881.000000,2281.000000,94.800000,12.500000,18.858308
max,2018.000000,3.608503e+10,0.750000,3.500000,4565.461538,84.300000,4.930000,2003.839184,100.000000,96.814672,82.526316,74.647887,78.300000,250001.000000,458625.000000,13033.000000,100.000000,83.900000,56.237817


<h3>Add bins column</h3>

In [31]:

bins_leaks_per_person = [
    "0-0.0015",
    "0.00151-0.003",
    "0.0031-0.0045",
    "0.00451-0.006",
    "0.0061-0.0075",
    "0.00751-0.009",
    "0.0091-10",
    "park_cemetery"
]

In [32]:
full_df['bins_leaks_per_person']=full_df['gas_leaks_per_person'].apply(lambda x: bins_leaks_per_person[0] if float(x) <=0.0015 else bins_leaks_per_person[1] if float(x) <=0.003 else bins_leaks_per_person[2] if float(x) <=0.0045 else bins_leaks_per_person[3] if float(x) <=0.006 else bins_leaks_per_person[4] if float(x) <=0.0075 else bins_leaks_per_person[5] if float(x) <=0.009 else bins_leaks_per_person[6] )

In [34]:
dfg['geoid']= dfg['geoid'].astype('int64')

<h3> Create geojson files according to bins</h3>

In [35]:
merged = dfg.merge(full_df, on='geoid' )

In [36]:
merged.head()

,geoid,ntaname,boro_name,shape_area,geometry,incident_year,gas_leaks_per_person,avg_assigned_fdny_engines#,avg_incident_duration,median_age,...,lonely_housholder_over65%,housholders_grandparents_responsible_for_grandchildren%,unemployed%,median_houshold_income,mean_houshold_income,total_housing_units,occupied_housing_units%,vacant_housing_units%,not_us_citizen%,bins_leaks_per_person
0,36085000900,west new brighton-new brighton-st. george,staten island,2497009.71359,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ...",2013,0.000486,2.0,1212.000000,32.6,...,1.434720,0.430416,11.3,60817.0,75868.0,918,75.9,24.1,16.431697,0-0.0015
1,36085000900,west new brighton-new brighton-st. george,staten island,2497009.71359,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ...",2014,0.004587,2.0,2006.333333,29.5,...,1.815431,0.756430,9.2,60352.0,68348.0,932,70.9,29.1,18.858308,0.00451-0.006
2,36085000900,west new brighton-new brighton-st. george,staten island,2497009.71359,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ...",2015,0.005376,2.1,1474.300000,30.0,...,3.343949,0.636943,11.0,52500.0,69225.0,919,68.3,31.7,17.580645,0.00451-0.006
3,36085000900,west new brighton-new brighton-st. george,staten island,2497009.71359,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ...",2016,0.003480,2.0,981.833333,31.7,...,8.292683,0.650407,11.6,47868.0,62529.0,881,69.8,30.2,16.531323,0.0031-0.0045
4,36085000900,west new brighton-new brighton-st. george,staten island,2497009.71359,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ...",2017,0.002846,2.0,754.400000,32.7,...,7.894737,0.000000,9.8,55250.0,69282.0,868,74.4,25.6,15.651679,0.00151-0.003


In [26]:
for index, row in merged.iterrows():
    if 'cemete' in row['ntaname']:
        merged.loc[index, 'bins_leaks_per_person'] = bins_leaks_per_person[7]

In [125]:
years = [2013 + x for x in range(6)]
for year in years:
    for bin in bins_leaks_per_person:
        print(bin)
        geo_layer = merged[(merged['bins_leaks_per_person'] == bin) & (merged['incident_year'] == year)] # geo

        try:
            geo_layer.to_file("C:/Users/mskac/machineLearning/GasLeakConEd/data/geolayers/{}.geojson".format(str(year)+'_'+bin), driver='GeoJSON')
        except:
            print('err')

0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery
0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery
0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery
0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery
0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery
0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery


In [26]:
full_df = pd.read_csv('https://raw.githubusercontent.com/MarinaOrzechowski/GasLeakConEd/timeline_branch/data/processed/important_(used_in_app)/Merged_asc_fdny_data.csv')

In [27]:
full_df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [28]:
full_df = full_df.dropna()

In [29]:

full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11556 entries, 0 to 11606
Data columns (total 19 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   incident_year                                            11556 non-null  int64  
 1   geoid                                                    11556 non-null  int64  
 2   gas_leaks_per_person                                     11556 non-null  float64
 3   avg_assigned_fdny_engines#                               11556 non-null  float64
 4   avg_incident_duration                                    11556 non-null  float64
 5   median_age                                               11556 non-null  float64
 6   avg_houshold_size                                        11556 non-null  float64
 7   avg_year_built                                           11556 non-null  float64
 8   nonfamily_housholds%      

In [30]:
full_df[full_df.gas_leaks_per_person > 0.003310].describe()

,incident_year,geoid,gas_leaks_per_person,avg_assigned_fdny_engines#,avg_incident_duration,median_age,avg_houshold_size,avg_year_built,nonfamily_housholds%,lonely_housholder%,lonely_housholder_over65%,housholders_grandparents_responsible_for_grandchildren%,unemployed%,median_houshold_income,mean_houshold_income,total_housing_units,occupied_housing_units%,vacant_housing_units%,not_us_citizen%
count,2889.000000,2.889000e+03,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000
mean,2015.771201,3.604885e+10,0.007172,2.062730,1392.592602,35.876843,2.542378,1949.817780,43.387317,35.088445,11.002267,2.224546,10.436622,57330.690550,82635.638629,1823.581516,90.168847,9.831326,14.606438
std,1.559281,2.503189e+07,0.020157,0.197013,378.556565,6.532454,0.526167,13.676322,16.276726,13.111069,6.958534,3.071707,6.525277,36490.440626,57392.139721,1238.439008,7.634382,7.634398,7.352100
min,2013.000000,3.600500e+10,0.003310,0.714286,629.250000,15.700000,1.030000,1922.479141,0.000000,0.000000,0.000000,0.000000,0.000000,9053.000000,11652.000000,16.000000,16.100000,0.000000,0.000000
25%,2015.000000,3.604702e+10,0.003935,1.960000,1151.666667,31.700000,2.210000,1940.160480,31.300161,26.112760,6.688778,0.000000,5.700000,30893.000000,45552.000000,1043.000000,87.500000,5.200000,9.407070
50%,2016.000000,3.604711e+10,0.004911,2.060000,1338.250000,34.900000,2.550000,1949.230565,41.403126,33.333333,9.860724,1.454898,9.300000,46810.000000,65599.000000,1555.000000,91.600000,8.400000,13.370474
75%,2017.000000,3.606102e+10,0.006981,2.166667,1539.035714,38.900000,2.900000,1957.333523,53.057065,42.257742,13.907285,3.426791,13.900000,74712.000000,98881.000000,2281.000000,94.800000,12.500000,18.858308
max,2018.000000,3.608503e+10,0.750000,3.500000,4565.461538,84.300000,4.930000,2003.839184,100.000000,96.814672,82.526316,74.647887,78.300000,250001.000000,458625.000000,13033.000000,100.000000,83.900000,56.237817


<h3>Add bins column</h3>

In [31]:

bins_leaks_per_person = [
    "0-0.0015",
    "0.00151-0.003",
    "0.0031-0.0045",
    "0.00451-0.006",
    "0.0061-0.0075",
    "0.00751-0.009",
    "0.0091-10",
    "park_cemetery"
]

In [32]:
full_df['bins_leaks_per_person']=full_df['gas_leaks_per_person'].apply(lambda x: bins_leaks_per_person[0] if float(x) <=0.0015 else bins_leaks_per_person[1] if float(x) <=0.003 else bins_leaks_per_person[2] if float(x) <=0.0045 else bins_leaks_per_person[3] if float(x) <=0.006 else bins_leaks_per_person[4] if float(x) <=0.0075 else bins_leaks_per_person[5] if float(x) <=0.009 else bins_leaks_per_person[6] )

In [34]:
dfg['geoid']= dfg['geoid'].astype('int64')

<h3> Create geojson files according to bins</h3>

In [35]:
merged = dfg.merge(full_df, on='geoid' )

In [36]:
merged.head()

,geoid,ntaname,boro_name,shape_area,geometry,incident_year,gas_leaks_per_person,avg_assigned_fdny_engines#,avg_incident_duration,median_age,...,lonely_housholder_over65%,housholders_grandparents_responsible_for_grandchildren%,unemployed%,median_houshold_income,mean_houshold_income,total_housing_units,occupied_housing_units%,vacant_housing_units%,not_us_citizen%,bins_leaks_per_person
0,36085000900,west new brighton-new brighton-st. george,staten island,2497009.71359,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ...",2013,0.000486,2.0,1212.000000,32.6,...,1.434720,0.430416,11.3,60817.0,75868.0,918,75.9,24.1,16.431697,0-0.0015
1,36085000900,west new brighton-new brighton-st. george,staten island,2497009.71359,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ...",2014,0.004587,2.0,2006.333333,29.5,...,1.815431,0.756430,9.2,60352.0,68348.0,932,70.9,29.1,18.858308,0.00451-0.006
2,36085000900,west new brighton-new brighton-st. george,staten island,2497009.71359,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ...",2015,0.005376,2.1,1474.300000,30.0,...,3.343949,0.636943,11.0,52500.0,69225.0,919,68.3,31.7,17.580645,0.00451-0.006
3,36085000900,west new brighton-new brighton-st. george,staten island,2497009.71359,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ...",2016,0.003480,2.0,981.833333,31.7,...,8.292683,0.650407,11.6,47868.0,62529.0,881,69.8,30.2,16.531323,0.0031-0.0045
4,36085000900,west new brighton-new brighton-st. george,staten island,2497009.71359,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ...",2017,0.002846,2.0,754.400000,32.7,...,7.894737,0.000000,9.8,55250.0,69282.0,868,74.4,25.6,15.651679,0.00151-0.003


In [26]:
for index, row in merged.iterrows():
    if 'cemete' in row['ntaname']:
        merged.loc[index, 'bins_leaks_per_person'] = bins_leaks_per_person[7]

In [125]:
years = [2013 + x for x in range(6)]
for year in years:
    for bin in bins_leaks_per_person:
        print(bin)
        geo_layer = merged[(merged['bins_leaks_per_person'] == bin) & (merged['incident_year'] == year)] # geo

        try:
            geo_layer.to_file("C:/Users/mskac/machineLearning/GasLeakConEd/data/geolayers/{}.geojson".format(str(year)+'_'+bin), driver='GeoJSON')
        except:
            print('err')

0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery
0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery
0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery
0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery
0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery
0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery


In [28]:

for bin in bins_leaks_per_person:
    print(bin)
    geo_layer = merged[(merged['bins_leaks_per_person'] == bin)] # geo

    try:
        geo_layer.to_file("C:/Users/mskac/machineLearning/GasLeakConEd/data/geolayers/{}.geojson".format('all'+'_'+bin), driver='GeoJSON')
    except:
        print('err')

0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery


In [28]:

for bin in bins_leaks_per_person:
    print(bin)
    geo_layer = merged[(merged['bins_leaks_per_person'] == bin)] # geo

    try:
        geo_layer.to_file("C:/Users/mskac/machineLearning/GasLeakConEd/data/geolayers/{}.geojson".format('all'+'_'+bin), driver='GeoJSON')
    except:
        print('err')

0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery


<h2> All years consolidated </h2>

In [41]:
full_df = pd.read_csv('https://raw.githubusercontent.com/MarinaOrzechowski/GasLeakConEd/timeline_branch/data/processed/important_(used_in_app)/Merged_asc_fdny_data_all_years.csv')

In [42]:
full_df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [43]:
full_df = full_df.dropna()

In [44]:

full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2039 entries, 0 to 2050
Data columns (total 17 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   geoid                                                    2039 non-null   int64  
 1   gas_leaks_per_person                                     2039 non-null   float64
 2   avg_assigned_fdny_engines#                               2039 non-null   float64
 3   avg_incident_duration                                    2039 non-null   float64
 4   median_age                                               2039 non-null   float64
 5   avg_houshold_size                                        2039 non-null   float64
 6   avg_year_built                                           2039 non-null   float64
 7   nonfamily_housholds%                                     2039 non-null   float64
 8   lonely_housholder%          

In [45]:
full_df[full_df.gas_leaks_per_person > 0.003310].describe()

,geoid,gas_leaks_per_person,avg_assigned_fdny_engines#,avg_incident_duration,median_age,avg_houshold_size,avg_year_built,nonfamily_housholds%,lonely_housholder%,lonely_housholder_over65%,housholders_grandparents_responsible_for_grandchildren%,unemployed%,mean_houshold_income,total_housing_units,occupied_housing_units%,vacant_housing_units%,not_us_citizen%
count,1.886000e+03,1886.000000,1886.000000,1886.000000,1886.000000,1886.000000,1886.000000,1886.000000,1886.000000,1886.000000,1886.000000,1886.000000,1886.000000,1886.000000,1886.000000,1886.000000,1886.000000
mean,3.605484e+10,0.017481,2.069748,1403.803385,36.882462,2.780977,1947.448356,36.303833,29.277258,9.997009,2.295076,9.208344,83568.951767,1645.573515,91.292246,8.707913,16.115292
std,2.529217e+07,0.031594,0.126280,332.277993,5.970679,0.572313,12.878092,15.075692,11.954781,5.618702,2.236107,4.586235,45714.992962,1111.134442,5.317954,5.317963,8.690516
min,3.600500e+10,0.003311,1.180328,790.470588,13.233333,1.060000,1922.166694,8.181547,5.552044,0.000000,0.000000,0.000000,14351.333333,30.500000,34.266667,0.000000,0.000000
25%,3.604703e+10,0.007254,2.000000,1184.720095,32.875000,2.395000,1937.667095,24.797580,20.476566,6.243929,0.510310,5.950000,56849.541667,919.250000,89.341667,5.487500,9.789386
50%,3.604712e+10,0.011694,2.071429,1365.707541,36.291667,2.780833,1946.043807,33.553222,27.759483,8.987020,1.750047,8.358333,74371.000000,1404.750000,92.200000,7.800000,14.431301
75%,3.608103e+10,0.019310,2.135135,1557.412281,40.500000,3.150000,1954.959935,45.480477,35.836867,12.553238,3.505419,11.450000,94656.333333,2017.000000,94.512500,10.658333,20.864277
max,3.608503e+10,0.987013,2.590000,7099.438596,75.116667,5.371667,2003.646228,98.830409,96.014642,76.547135,22.902307,57.200000,438563.500000,12857.166667,100.000000,65.733333,53.846503


<h3>Add bins column</h3>

In [46]:

bins_leaks_per_person = [
    "0-0.0015",
    "0.00151-0.003",
    "0.0031-0.0045",
    "0.00451-0.006",
    "0.0061-0.0075",
    "0.00751-0.009",
    "0.0091-10",
    "park_cemetery"
]

In [47]:
full_df['bins_leaks_per_person']=full_df['gas_leaks_per_person'].apply(lambda x: bins_leaks_per_person[0] if float(x) <=0.0015 else bins_leaks_per_person[1] if float(x) <=0.003 else bins_leaks_per_person[2] if float(x) <=0.0045 else bins_leaks_per_person[3] if float(x) <=0.006 else bins_leaks_per_person[4] if float(x) <=0.0075 else bins_leaks_per_person[5] if float(x) <=0.009 else bins_leaks_per_person[6] )

In [48]:
dfg['geoid']= dfg['geoid'].astype('int64')

<h3> Create geojson files according to bins</h3>

In [49]:
merged = dfg.merge(full_df, on='geoid' )

In [50]:
merged.head()

,geoid,ntaname,boro_name,shape_area,geometry,gas_leaks_per_person,avg_assigned_fdny_engines#,avg_incident_duration,median_age,avg_houshold_size,...,lonely_housholder%,lonely_housholder_over65%,housholders_grandparents_responsible_for_grandchildren%,unemployed%,mean_houshold_income,total_housing_units,occupied_housing_units%,vacant_housing_units%,not_us_citizen%,bins_leaks_per_person
0,36085000900,west new brighton-new brighton-st. george,staten island,2497009.71359,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ...",0.020919,2.025641,1359.461538,31.566667,2.845000,...,24.889657,5.390057,0.534915,10.100000,69233.000000,896.666667,73.033333,26.966667,16.749541,0.0091-10
1,36061009800,turtle bay-east midtown,manhattan,1906016.35002,"MULTIPOLYGON (((-73.96433 40.75638, -73.96479 ...",0.021768,2.018868,1980.578616,40.016667,1.700000,...,56.306292,15.509536,0.000000,2.616667,199916.666667,5275.500000,80.400000,19.600000,7.952596,0.0091-10
2,36061010000,turtle bay-east midtown,manhattan,1860938.37721,"MULTIPOLYGON (((-73.96802 40.75958, -73.96848 ...",0.055196,2.010204,2186.142857,36.216667,1.488333,...,67.174451,12.098914,0.000000,3.483333,154906.333333,1375.333333,86.083333,13.916667,14.480270,0.0091-10
3,36061010200,midtown-midtown south,manhattan,1860992.70485,"MULTIPOLYGON (((-73.97124 40.76094, -73.97170 ...",0.109817,2.045455,1919.363636,40.250000,1.898333,...,31.914907,5.188030,0.000000,0.000000,140282.666667,294.833333,34.266667,65.733333,15.921674,0.0091-10
4,36061010400,midtown-midtown south,manhattan,1864600.37503,"MULTIPOLYGON (((-73.97446 40.76229, -73.97491 ...",0.038809,2.105263,2355.657895,53.233333,1.573333,...,57.171285,24.372617,0.000000,6.900000,183816.666667,1046.166667,59.083333,40.916667,16.808012,0.0091-10


In [51]:
for index, row in merged.iterrows():
    if 'cemete' in row['ntaname']:
        merged.loc[index, 'bins_leaks_per_person'] = bins_leaks_per_person[7]

In [53]:
for bin in bins_leaks_per_person:
    print(bin)
    geo_layer = merged[merged['bins_leaks_per_person'] == bin] # geo

    try:
        geo_layer.to_file("C:/Users/mskac/machineLearning/GasLeakConEd/data/geolayers/{}.geojson".format('all'+'_'+bin), driver='GeoJSON')
    except:
        print('err')

0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
park_cemetery
